## dataset 준비
dataset을 negative item을 추가해서 준비한다.

1. outfit을 구성하는 item의 갯수 중 50퍼센트가 넘지않게 item을 랜덤하게 선택한다.
2. 해당 item과 같은 category의 다른 item을 negative item으로 대체한다.
3. negative item이라 하면은 outfit의 나머지 item들과 다른 outfit에서도 함께 등장하지 않은 item을 뜻한다.

In [94]:
import os
import numpy as np
import json
from matplotlib import pyplot as plt
from collections import defaultdict     ### dictionary의 value에 list를 추가하고싶을 때 이 라이브러리를 쓰면됨
from random import *
import secrets
import random
import time

## Json data load

train data는 17316개의 outfit으로 구성되어 있다.
<br/>
valid data는 1497개의 outfit으로 구성되어 있다.
<br/>
test data는 3076개의 outfit으로 구성되어 있다.

In [95]:
### json, image의 path
json_path = './polyvore-dataset-master/'
image_path = json_path + 'images/'

### train, valid, testset의 json read
with open(json_path + 'train_no_dup.json', 'r') as f:
    train_data = json.load(f)
    
with open(json_path + 'valid_no_dup.json', 'r') as f:
    valid_data = json.load(f)

with open(json_path + 'test_no_dup.json', 'r') as f:
    test_data = json.load(f)

### data 구조 확인
print('train_data shape: {}'.format(np.shape(train_data)))
print('valid_data shape: {}'.format(np.shape(valid_data)))
print('test_data shape:  {}'.format(np.shape(test_data)))

train_data shape: (17316,)
valid_data shape: (1497,)
test_data shape:  (3076,)


### Outfit을 구성하는 item의 갯수 중 50% 넘지 않게 item을 random하게 선택한다.

* 각 item 별로 등장하는 outfit을 찾는다.

In [96]:
### item별로 등장하는 outfit을 찾는다.
### defaultdict()를 사용하면 dictionary의 value에 list를 넣기 쉬움
train_item2outfit = defaultdict(list)
valid_item2outfit = defaultdict(list)
test_item2outfit  = defaultdict(list)

### train data
for i in range(len(train_data)):
    list_ids = []
    ### 각 outfit에 있는 item들을 list에 저장한다.
    for j in range(len(train_data[i]['items'])):
        _, ids = train_data[i]['items'][j]['image'].split('id=')
        list_ids.append(ids)
    ### 저장한 list의 item이 등장하는 outfit을 붙인다.
    for j in range(len(list_ids)):
        train_item2outfit[list_ids[j]].append(train_data[i]['set_id'])

### valid data
for i in range(len(valid_data)):
    list_ids = []
    for j in range(len(valid_data[i]['items'])):
        _, ids = valid_data[i]['items'][j]['image'].split('id=')
        list_ids.append(ids)
    for j in range(len(list_ids)):
        valid_item2outfit[list_ids[j]].append(valid_data[i]['set_id'])

### test data
for i in range(len(test_data)):
    list_ids = []
    for j in range(len(test_data[i]['items'])):
        _, ids = test_data[i]['items'][j]['image'].split('id=')
        list_ids.append(ids)
    for j in range(len(list_ids)):
        test_item2outfit[list_ids[j]].append(test_data[i]['set_id'])


data를 저장해서 확인해본다.
<br/>
key: item_id, value: outfit_list

In [97]:
save_path = './negative_outfit/'

with open(save_path + 'train_item2outfit.json', 'w') as f:
    json.dump(train_item2outfit, f, indent=4)

with open(save_path + 'valid_item2outfit.json', 'w') as f:
    json.dump(valid_item2outfit, f, indent=4)

with open(save_path + 'test_item2outfit.json', 'w') as f:
    json.dump(test_item2outfit, f, indent=4)

outfit에 있는 item을 절반이 넘지 않게 랜덤하게 선택해본다.
<br/>
먼저 outfit 별로 갖고 있는 item을 확인할 수 있도록 간략하게 json file을 만들어본다.

In [98]:
### outfit에 등장하는 item들을 저장한다.
### defaultdict()를 사용하면 dictionary의 value에 list를 넣기 쉬움
train_outfit2item = defaultdict(list)
valid_outfit2item = defaultdict(list)
test_outfit2item  = defaultdict(list)

train_set_ids = []
valid_set_ids = []
test_set_ids  = []

### train data
for i in range(len(train_data)):
    train_list_ids = []
    set_id = train_data[i]['set_id']
    train_set_ids.append(set_id)
    for j in range(len(train_data[i]['items'])):
        _, ids = train_data[i]['items'][j]['image'].split('id=')
        train_list_ids.append(ids)

    train_outfit2item[set_id] = train_list_ids

### valid data
for i in range(len(valid_data)):
    valid_list_ids = []
    set_id = valid_data[i]['set_id']
    valid_set_ids.append(set_id)
    for j in range(len(valid_data[i]['items'])):
        _, ids = valid_data[i]['items'][j]['image'].split('id=')
        valid_list_ids.append(ids)

    valid_outfit2item[set_id] = valid_list_ids

### test data
for i in range(len(test_data)):
    test_list_ids = []
    set_id = test_data[i]['set_id']
    test_set_ids.append(set_id)
    for j in range(len(test_data[i]['items'])):
        _, ids = test_data[i]['items'][j]['image'].split('id=')
        test_list_ids.append(ids)

    test_outfit2item[set_id] = test_list_ids

In [99]:
with open(save_path + 'train_outfit2item.json', 'w') as f:
    json.dump(train_outfit2item, f, indent=4)

with open(save_path + 'valid_outfit2item.json', 'w') as f:
    json.dump(valid_outfit2item, f, indent=4)

with open(save_path + 'test_outfit2item.json', 'w') as f:
    json.dump(test_outfit2item, f, indent=4)

outfit 내에서 item을 랜덤하게 50%가 넘지 않게 선택한다. <br/>
이 때, 이 item을 대체할 item은 다른 outfit에서 한번도 같이 등장한 적이 없어야한다.

In [100]:
train_choice_items = defaultdict(list)
valid_choice_items = defaultdict(list)
test_choice_items = defaultdict(list)

### train data에 대하여
for i in range(len(train_data)):
    ### rand_num: 각 outfit 별로 random하게 선택할 item의 갯수
    set_id = train_data[i]['set_id']
    length = len(train_outfit2item[train_set_ids[i]])//2
    rand_num = np.random.randint(1, length)
    # print(rand_num)
    
    ### random한 item을 선택한다.
    choice = random.choices(train_outfit2item[train_set_ids[i]], k=rand_num)
    train_choice_items[set_id] = choice

### valid data에 대하여
for i in range(len(valid_data)):
    set_id = valid_data[i]['set_id']
    length = len(valid_outfit2item[valid_set_ids[i]])//2
    rand_num = np.random.randint(1, length)

    choice = random.choices(valid_outfit2item[valid_set_ids[i]], k=rand_num)
    valid_choice_items[set_id] = choice

### test data에 대하여
for i in range(len(test_data)):
    set_id = test_data[i]['set_id']
    length = len(test_outfit2item[test_set_ids[i]])//2
    rand_num = np.random.randint(1, length)

    choice = random.choices(test_outfit2item[test_set_ids[i]], k=rand_num)
    test_choice_items[set_id] = choice

In [101]:
# key: outfit id, value: negative item으로 선택될 n개의 items
with open(save_path + 'train_choice_item.json', 'w') as f:
    json.dump(train_choice_items, f, indent=4)

with open(save_path + 'valid_choice_item.json', 'w') as f:
    json.dump(valid_choice_items, f, indent=4)

with open(save_path + 'test_choice_item.json', 'w') as f:
    json.dump(test_choice_items, f, indent=4)

negative item의 후보로 선택된 item을 다른 item으로 넣어준다. <br/>
이 때, 다른 item은 선택된 item과 같은 category여야한다. <br/>
또한, 선택되지 않은 나머지 item들과 어떠한 outfit에서도 함께 등장하지 아니한다.

In [102]:
with open(save_path + 'train_choice_item.json', 'r') as f:
    train_choice = json.load(f)
    
with open(save_path + 'valid_choice_item.json', 'r') as f:
    valid_choice = json.load(f)

with open(save_path + 'test_choice_item.json', 'r') as f:
    test_choice = json.load(f)

with open(save_path + 'all_item2outfit.json', 'r') as f:
    item2outfit = json.load(f)

with open(save_path + 'all_outfit2item.json', 'r') as f:
    outfit2item = json.load(f)

In [ ]:
for i in range(len(train_choice)):
    

생각해보니까 train, valid, test set에 중복되는 item들이 있음 <br/>
그렇기 때문에 한번에 다 넣어서 찾아야함 <br/>
하지만 dataset을 소개한 논문을 확인해본 결과, 중복되는 item은 없음

In [61]:
# # from itertools import chain
# # from collections import defaultdict

# # tmp_outfit2item = defaultdict(list)
# # all_outfit2item = defaultdict(list)

# # for k, v in chain(train_outfit2item.items(), valid_outfit2item.items()):
# #     tmp_outfit2item[k].append(v)

# # for k, v in chain(tmp_outfit2item.items(), test_outfit2item.items()):
# #     all_outfit2item[k].append(v)

# # tmp_item2outfit = defaultdict(list)
# # all_item2outfit = defaultdict(list)

# # for k, v in chain(train_item2outfit.items(), valid_item2outfit.items()):
# #     tmp_item2outfit[k].append(v)

# # for k, v in chain(tmp_item2outfit.items(), test_item2outfit.items()):
# #     all_item2outfit[k].append(v)

# train_outfit2item.update(valid_outfit2item)
# train_outfit2item.update(test_outfit2item)

# train_item2outfit.update(valid_item2outfit)
# train_item2outfit.update(test_item2outfit)

# all_outfit2item = train_outfit2item
# all_item2outfit = train_item2outfit

In [60]:
# with open(save_path + 'all_outfit2item.json', 'w') as f:
#     json.dump(all_outfit2item, f, indent=4)

# with open(save_path + 'all_item2outfit.json', 'w') as f:
#     json.dump(all_item2outfit, f, indent=4)